## Introducción

Frecuentemente, los datos estarán dispersos en varios archivos o bases de datos o estar organizados de forma que no sea fácil de analizar. Por este motivo, `Pandas` incorpora métodos que faciliten esta tarea.

En esta clase vamos a ver las herramientas para ayudar a combinar, unir y reorganizar los datos.

Antes, es necesario repasar el concepto de indexación múltiple e indexación jerárquica.

### Indexing jerárquico

La indexación jerárquica es una característica importante de `Pandas` que permite tener múltiples (dos o más) niveles de índice en un eje. De forma algo abstracta, proporciona una forma de trabajar con datos de dimensiones superiores en una forma de dimensiones inferiores. 

Comencemos con un ejemplo simple: crear una serie con una lista de listas como `index`

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Armamos una serie con doble indice. Es decir dos coordenadas por cada valor. 
series1 = pd.Series(np.random.randn(9),
                   index=[['a','a','a','b','b','c','c','d','d'],[1,2,3,1,3,1,2,2,3]])

display(series1)

a  1    0.683364
   2    0.928215
   3    0.586794
b  1   -0.107546
   3   -2.150267
c  1    0.177161
   2    0.077045
d  2   -1.281147
   3   -0.283505
dtype: float64

Lo que estás viendo es una vista bonita de una serie con a como su índice. Los "huecos" en la pantalla del índice significan "use la etiqueta directamente encima":`MultiIndex`

In [4]:
# si accedemos al atributo index de esta serie vemos que es Multiindex y los elementos son Tuplas y podemos tener índices con la cantidad que 
# necesitemos de elementos.
series1.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [16]:
# Vemos como acceder a un elemento
series1.loc[('a',2)]

0.4286073872552598

Con un objeto indexado jerárquicamente, es posible la llamada indexación parcial, lo que le permite seleccionar de forma concisa subconjuntos de los datos:

In [18]:
# Por ejemplo, sabemos que hay mas de un elemento para el índice b

series1['b']


1   -1.966401
3    2.023618
dtype: float64

In [19]:
# Podemos hacer indexación por slicing
series1['b':'c']

b  1   -1.966401
   3    2.023618
c  1    0.561608
   2    0.080730
dtype: float64

La selección es posible incluso desde un nivel "interno". Aquí selecciono todos los valores que tienen el valor del segundo nivel de índice: `2`

In [20]:
# Podemos traer todo el primer índice y un valor particular del segundo
series1.loc[:,2]

a    0.428607
c    0.080730
d   -1.631250
dtype: float64

In [7]:
# Podemos último si queremos acceder solo a algunos elementos pero utilizando sus coordenadas:
series1.loc[['a','b'],[1,3]]

a  1    0.683364
   3    0.586794
b  1   -0.107546
   3   -2.150267
dtype: float64

***Método unstack***

La indexación jerárquica desempeña un papel importante en la remodelación de los datos y en las operaciones basadas en grupos, como la formación de una tabla dinámica. Por ejemplo, puede reorganizar estos datos en un DataFrame utilizando su método: `unstack`

In [22]:
# Recordemos nuestra serie.
series1

a  1    1.725775
   2    0.428607
   3   -2.239050
b  1   -1.966401
   3    2.023618
c  1    0.561608
   2    0.080730
d  2   -1.631250
   3    0.361573
dtype: float64

In [23]:
# Con este método podemos tomar la serie del indice que está mas adentro y convertirlar en columna. 
# Es una forma de pivotear sobre la serie. Notar que completa las posiciones donde no hay nada con nulos.
series1.unstack()

,1,2,3
a,1.725775,0.428607,-2.239050
b,-1.966401,NaN,2.023618
c,0.561608,0.080730,NaN
d,NaN,-1.631250,0.361573


La operación inversa de es : `unstack stack`

In [8]:
# El método stack hace la operatoria inversa, si concatenamos ambas para trabajar sobre la serie pivoteada, volvemos
# al formato original.
series1.unstack().stack()

a  1    0.683364
   2    0.928215
   3    0.586794
b  1   -0.107546
   3   -2.150267
c  1    0.177161
   2    0.077045
d  2   -1.281147
   3   -0.283505
dtype: float64

Con un `DataFrame`, cualquiera de los ejes puede tener un índice jerárquico:

In [9]:
# En este caso podemos tener un multi índex o pindice jerárquico tanto en filas como en columnas.
df1 = pd.DataFrame(np.arange(12).reshape((4,3)),
                   index=[['a','a','b','b'],[1,2,1,2]],
                   columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])

display(df1)

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

Los niveles jerárquicos pueden tener nombres (como `cadenas` o cualquier objeto de Python). Si es así, estos aparecerán en la salida de la consola:

In [29]:
# Damos nombre a los índices.
df1.index.names = ['key1','key2']
df1.columns.names = ['state','color']
display(df1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Estos nombres sustituyen al atributo, que solo se utiliza con índices de un solo nivel `.name`

Podemos ver cuántos niveles tiene un índice accediendo a su atributo: `nlevels`

In [10]:
df1.index.nlevels

2

Con la indexación parcial de columnas, puede seleccionar de manera similar grupos de columnas:

In [11]:
# en este caso nos traemos lo relacionado con ohio
df1['Ohio']

Green  Red
a 1      0    1
  2      3    4
b 1      6    7
  2      9   10

Puede crearse por sí mismo y luego reutilizarse; las columnas del DataFrame anterior con nombres de nivel también se pueden crear de la siguiente manera: `MultiIndex`

In [12]:
pd.MultiIndex.from_arrays([["Ohio", "Ohio", "Colorado"],
                          ["Green", "Red", "Green"]],
                          names=["state", "color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Re-organización y Re-sorting de indices

A veces será necesario reorganizar el orden de los niveles en un eje u ordenar los datos por los valores en un nivel específico. El método `swaplevel()` toma dos números o nombres de nivel y devuelve un nuevo objeto con los niveles intercambiados (pero por lo demás los datos no se modifican):

In [31]:
# El metodo swaplevel trabaja a nivel índice y no modifica nada mas.
df1.swaplevel('key1','key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

`sort_index` De forma predeterminada, ordena los datos lexicográficamente utilizando todos los niveles de índice, pero puede optar por usar solo un único nivel o un subconjunto de niveles para ordenar pasando el argumento:

In [13]:
# Ordenamos por nivel en Key1
df1.sort_index(level = 0,ascending = True)

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [14]:
# Ordenamos por nivel
df1.sort_index(level = 1,ascending = True)

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
b 1     6   7        8
a 2     3   4        5
b 2     9  10       11

In [15]:
# Probamos de combinar el swap y el sort_index
df1.swaplevel(0,1).sort_index(level = 0)

Ohio     Colorado
    Green Red    Green
1 a     0   1        2
  b     6   7        8
2 a     3   4        5
  b     9  10       11

El uso de índices simples y múltiples facilita el acceso a los datos utilizando métodos como `loc` e `iloc`.

También son muy utilizados para combinar múltiples fuentes de datos, relacionándolos mediante el `index`

### Combinación y unión de datasets

Para agregar información a un dataset (`DataFrame`), se pueden incluir filas o columnas. 
Dentro del módulo `Pandas` existen 3 métodos para combinar `DataFrames`:
 - `pandas.join()` conecta filas de `DataFrames` alineando el índice o alguna columna de uno con el índice del otro. Esto será familiar para los usuarios de SQL u otras bases de datos relacionales, ya que implementa operaciones de `join` de bases de datos.
 - `pandas.merge()` conecta filas en `DataFrames` utilizando columnas o índices para alinearlos. Este método es muy similar al `join`, pero es mas versatil, ya que permite relacionar `DataFrames` utilizando columnas diferentes. 
 - `pandas.concat()` concatena o "apila" objetos a lo largo de un eje. Este método permite actualizar los indices o manterlos al concatenar.
 
 Veamos un pequeño ejemplo de como se utilizan estas funciones.

In [19]:
df2 = pd.DataFrame({'key':['b','b','a','c','a','a','b'],
                   'data1':range(7)})

df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [17]:
df3 = pd.DataFrame({'key':['a','b','d'],
                   'data2':range(3)})

df3

,key,data2
0,a,0
1,b,1
2,d,2


In [20]:
# how le dice que estructura preserva y le agregamos sufijos a las columnas que sean comunes a ambos 
df2.join(df3, how='left',lsuffix='_left', rsuffix = '_right')

,key_left,data1,key_right,data2
0,b,0,a,0.0
1,b,1,b,1.0
2,a,2,d,2.0
3,c,3,NaN,NaN
4,a,4,NaN,NaN
5,a,5,NaN,NaN
6,b,6,NaN,NaN


In [21]:
df2.join(df3, how='right',lsuffix='_left', rsuffix = '_right')

,key_left,data1,key_right,data2
0,b,0,a,0
1,b,1,b,1
2,a,2,d,2


Este es un ejemplo de una unión de muchos a uno; los datos de **df2** tienen múltiples filas etiquetadas como **a** y **b**, mientras que **df3** tiene sólo una fila para cada valor en la columna clave. Llamando a `merge()` con estos objetos obtenemos: 

In [44]:
# Ejemplo de uso de merge
pd.merge(df2,df3)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Hay que notar que no especificamos sobre cuál columna realizar el `merge()`. Si no se especifica esa información, el método `pd.merge()` utiliza los nombres de las columnas superpuestas como claves. Sin embargo, es una buena práctica especificar explícitamente

In [45]:
# Siempre que se pueda debemos ser bien explicitos.
pd.merge(df2,df3,on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Vemos otro ejemplo. Si los nombres de las columnas son diferentes en cada objeto, podemos especificarlos por separado:

In [24]:
df4 = pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],
                   'data1':range(7)})

df5 = pd.DataFrame({'rkey':['a','b','d'],
                   'data2': range(3)})
display(df4)
display(df5)

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,rkey,data2
0,a,0
1,b,1
2,d,2


In [25]:
# Le podemos decir que columna mirar en cada DF cuando tienen diferente nombre.
pd.merge(df4,df5, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


Podemos notar que los valores 'c' y 'd' y los datos asociados faltan en el resultado. Por default, `pd.merge()` realiza un `'inner'` join. Las claves del resultado son la intersección, o el conjunto común que se encuentra en ambas tablas. Otras opciones posibles son `'left'`, `'right'` y `'outer'`. El  `'outer'` join toma la unión de las claves, combinando el efecto de aplicar ambas uniones, `'left'` y `'right'`:

In [54]:
# Si le pasamos outer va a hacer la unión en lugar de la intersección
pd.merge(df2, df3, how = 'outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


### Ejemplo con el dataset de producción:

In [28]:
user_usage = pd.read_csv("data2/user_usage.csv")
user_device = pd.read_csv("data2/user_device.csv")
devices = pd.read_csv("data2/android_devices.csv")
devices.rename(columns={'Retail Branding':'fabricante'},inplace=True)

In [29]:
user_usage.head()

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [30]:
user_device.head()

,id_uso,id_usuario,plataforma,plataforma_version,dispositivo,id_tipo_uso
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [31]:
devices.head(10)

,fabricante,Marketing Name,Dispositivo,Modelo,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,AD681H,Smartfren Andromax AD681H,NaN,NaN,NaN,NaN
1,NaN,NaN,FJL21,FJL21,NaN,NaN,NaN,NaN
2,NaN,NaN,T31,Panasonic T31,NaN,NaN,NaN,NaN
3,NaN,NaN,hws7721g,MediaPad 7 Youth 2,NaN,NaN,NaN,NaN
4,3Q,OC1020A,OC1020A,OC1020A,NaN,NaN,NaN,NaN
5,7Eleven,IN265,IN265,IN265,NaN,NaN,NaN,NaN
6,A.O.I. ELECTRONICS FACTORY,A.O.I.,TR10CS1_11,TR10CS1,NaN,NaN,NaN,NaN
7,AG Mobile,AG BOOST 2,BOOST2,E4010,NaN,NaN,NaN,NaN
8,AG Mobile,AG Flair,AG_Flair,Flair,NaN,NaN,NaN,NaN
9,AG Mobile,AG Go Tab Access 2,AG_Go_Tab_Access_2,AG_Go_Tab_Access_2,NaN,NaN,NaN,NaN


### Primer Merge

Intentemos analizar que consumos existen para cada tipo de dispositivo diferente. Para esto necesitamos realcionar mediante el código de usuario `id_uso` las tablas `user_usage` y `user_device`.

In [32]:
result = pd.merge(user_usage,
                 user_device[['id_uso','plataforma','dispositivo']],
                 on='id_uso')
result.head()

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso,plataforma,dispositivo
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


En este merge se pueden ver los datos de ambas tablas, unidos por la columna `id_uso`.
Analicemos en profundidad que es lo que sucedió durante el merge.

In [33]:
print("user_usage dimensions:{}".format(user_usage.shape))
print("user_device dimensions: {}".format(user_device[['id_uso','plataforma','dispositivo']].shape))

user_usage dimensions:(240, 4)
user_device dimensions: (272, 3)


In [34]:
# Si nos paramos en la primer tabla y vemos cuantas id_uso están presentes en el segundo DF
user_usage['id_uso'].isin(user_device['id_uso']).value_counts()

id_uso
True     159
False     81
Name: count, dtype: int64

Vemos que la cantidad de datos no es la misma. Esto se debe a que se realizó un `inner join`. Esto significa que las **claves** que no se encuentran en ambas tablas, se descartan.

In [35]:
result.shape

(159, 6)

#### Ejemplo Left Merge

El `left merge` o `left join` permite que se conserven todas las filas de uno de los 2 `DataFrames`.
En este caso, la tabla de la izquierda es la que se llama al método `merge` y la tabla de la derecha (`right`) es la que se utiliza como argumento de la función.
En este caso, `left=user_usage` y `right=user_device`.
Aplicando un left join, las columnas de la tabla derecha contendra `NaN` en todas las filas correspondientes a claves que no tienen su par en ambas tablas.

In [72]:
result = pd.merge(user_usage, 
                  user_device[['id_uso', 'plataforma', 'dispositivo']],
                  on = 'id_uso', how = 'left')

print("user_usage dimensions: {}".format(user_usage.shape))
print("result dimensions: {}".format(result.shape))

print(" Hay {} valores faltantes en el resultado.".format(result['dispositivo'].isnull().sum()))

user_usage dimensions: (240, 4)
result dimensions: (240, 6)
 Hay 81 valores faltantes en el resultado.


In [70]:
# Veo una muestra del resultado
result.head()

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso,plataforma,dispositivo
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F


In [71]:
result.tail()

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso,plataforma,dispositivo
235,260.66,68.44,896.96,25008,NaN,NaN
236,97.12,36.50,2815.00,25040,NaN,NaN
237,355.93,12.37,6828.09,25046,NaN,NaN
238,632.06,120.46,1453.16,25058,NaN,NaN
239,488.70,906.92,3089.85,25220,NaN,NaN


#### Ejemplo Right merge

El `right merge` o `right join` entre 2 `DataFrames` mantiene todas las filas correspondientes al `DataFrame` de la derecha, mientras que las columnas de la tabla izquierda de los registros que se encuentren en la tabla derecha, pero no en la izquierda, se completaran con `NaN`.

In [36]:
result = pd.merge(user_usage,
                 user_device[['id_uso', 'plataforma', 'dispositivo']],
                 on='id_uso', how='right')

print("user_device dimensions: {}".format(user_device.shape))
print("result dimensions: {}".format(result.shape))
print("Hay {} valores faltantes en la columna 'data_mb_mes' del resultado.".format(
        result['data_mb_mes'].isnull().sum()))
print("Hay {} valores faltantes en la columna 'plataforma' del resultado.".format(
        result['plataforma'].isnull().sum()))

user_device dimensions: (272, 6)
result dimensions: (272, 6)
Hay 113 valores faltantes en la columna 'data_mb_mes' del resultado.
Hay 0 valores faltantes en la columna 'plataforma' del resultado.


#### Ejemplo Outer merge

Un `full outer join` o `outer merge` mantiene todos los registros de ambos `DataFrames` en el resultado.
Las filas se alinearan en donde se compartan claves, y el resto de los registros tendran nulos en las columnas del `DataFrame` que corresponda.

En el resultado final, un subset de filas no tendran valores faltantes. Estos registros, que encontraron un match en la clave del otro `DataFrame` corresponden al resultado obrenido en el `inner merge`.

In [80]:
print("Hay {} valores únicos para ese id_uso en los DataFrames".format(
    pd.concat([user_usage['id_uso'],user_device['id_uso']]).unique().shape[0]))

result = pd.merge(user_usage,
                 user_device[['id_uso','plataforma','dispositivo']],
                 on='id_uso',how='outer',indicator=True)

print("El outer merge tiene {} registros".format(result.shape))

print("Hay {} filas sin valores faltantes".format(
    (result.apply(lambda x:x.isnull().sum,axis=1)==0).sum()))

Hay 353 valores únicos para ese id_uso en los DataFrames
El outer merge tiene (353, 7) registros
Hay 0 filas sin valores faltantes


A continuación se muestra una imagen que representa los distintos `joins` mediante diagramas de Venn.

<img src="clase 11/joins.jpg" width="500">


#### Merge Final - sumarizando productores de dispositivos

In [38]:
# First, add the platform and device to the user usage.
result = pd.merge(user_usage,
                 user_device[['id_uso', 'plataforma', 'dispositivo']],
                 on='id_uso',
                 how='left')

In [39]:
result

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso,plataforma,dispositivo
0,21.97,4.82,1557.33,22787,android,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F
...,...,...,...,...,...,...
235,260.66,68.44,896.96,25008,NaN,NaN
236,97.12,36.50,2815.00,25040,NaN,NaN
237,355.93,12.37,6828.09,25046,NaN,NaN
238,632.06,120.46,1453.16,25058,NaN,NaN


In [41]:
devices.sample(3)

,fabricante,Marketing Name,Dispositivo,Modelo,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
8135,Oppo,A1603,A1603,A1603,NaN,NaN,NaN,NaN
10863,Samsung,ProXpress M4580,fiber-athena,samsung-printer-tablet,NaN,NaN,NaN,NaN
5582,LGE,LG F70,f70n,LG-F370S,NaN,NaN,NaN,NaN


In [42]:
# Dispositivo y modelo tienen los mismos valores por eso le indicamos que los cruce por ahí
result = pd.merge(result,
                 devices[['fabricante','Modelo']],
                 left_on='dispositivo',
                 right_on='Modelo',
                 how='left')
result.head()

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso,plataforma,dispositivo,fabricante_x,Modelo_x,fabricante_y,Modelo_y
0,21.97,4.82,1557.33,22787,android,GT-I9505,Samsung,GT-I9505,Samsung,GT-I9505
1,1710.08,136.88,7267.55,22788,android,SM-G930F,Samsung,SM-G930F,Samsung,SM-G930F
2,1710.08,136.88,7267.55,22789,android,SM-G930F,Samsung,SM-G930F,Samsung,SM-G930F
3,94.46,35.17,519.12,22790,android,D2303,Sony,D2303,Sony,D2303
4,71.59,79.26,1557.33,22792,android,SM-G361F,Samsung,SM-G361F,Samsung,SM-G361F


#### Calculando estadísticas sobre el resultado final

Habiendo realizado los merges, se pueden calcular las estadisticas realizando un `groupby` sobre el manufacturador del dispositivo.

In [83]:
# Sobre este resultado final podemos hacer algunos cálculos. 
result.groupby("fabricante").agg({
    "min_saliente_mes":"mean",
    "sms_saliente_mes":"mean",
    "data_mb_mes":"mean",
    "id_uso":"count"
})

,min_saliente_mes,sms_saliente_mes,data_mb_mes,id_uso
fabricante,,,,
HTC,299.842955,93.059318,5144.077955,47
Huawei,81.526667,9.500000,1561.226667,6
LGE,111.530000,12.760000,1557.330000,3
Lava,60.650000,261.900000,12458.670000,2
Lenovo,215.920000,12.930000,1557.330000,2
Motorola,95.127500,65.666250,3946.500000,16
OnePlus,354.855000,48.330000,6575.410000,12
Samsung,191.010093,92.390463,4017.318889,126
Sony,177.315625,40.176250,3212.000625,16
